In [ ]:
# PHASE 0 — Kaggle SAFE Setup (NO NumPy downgrade)
# In Kaggle, Turn On Internet, Accelerator - GPU T4 ×2, Environmental Preferences - Pin to Original Environment
# Install only what YOLO needs
!pip install -U ultralytics kagglehub opencv-python-headless

import torch
import numpy as np
import cv2
from ultralytics import YOLO

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("NumPy:", np.__version__)
print("OpenCV:", cv2.__version__)

from ultralytics import YOLO
import torchvision
import numpy
import os, glob, shutil, zipfile

print("YOLO imported")
print("TorchVision:", torchvision.__version__)
print("NumPy:", numpy.__version__)



Torch: 2.8.0+cu126
CUDA available: True
NumPy: 2.0.2
OpenCV: 4.12.0
YOLO imported
TorchVision: 0.23.0+cu126
NumPy: 2.0.2


In [11]:
# PHASE 1 — Dataset Download (KaggleHub)
import kagglehub
import os
import shutil

# Download dataset
DATASET_ROOT = kagglehub.dataset_download("anulayakhare/crackathon-data")
print("Dataset downloaded at:", DATASET_ROOT)

# Actual dataset location (confirmed)
DATASET_PATH = os.path.join(DATASET_ROOT, "randomized_dataset")

# Required working directory structure
WORKING_PATH = "/kaggle/working/crackathon-data/randomized_dataset"
os.makedirs(WORKING_PATH, exist_ok=True)

# Copy train, val, test folders
for split in ["train", "val", "test"]:
    src = os.path.join(DATASET_PATH, split)
    dst = os.path.join(WORKING_PATH, split)

    if not os.path.exists(src):
        raise FileNotFoundError(f"❌ Source not found: {src}")

    if not os.path.exists(dst):
        shutil.copytree(src, dst)

    print(f"✅ {split} copied successfully")

print("🎉 Dataset copied to:", WORKING_PATH)


Dataset downloaded at: /kaggle/input/
✅ train copied successfully
✅ val copied successfully
✅ test copied successfully
🎉 Dataset copied to: /kaggle/working/crackathon-data/randomized_dataset


In [12]:
# PHASE 2 — data.yaml (Kaggle-safe)
DATA_YAML = "/kaggle/working/crackathon-data/data.yaml"

yaml_content = f"""
path: /kaggle/working/crackathon-data/randomized_dataset
train: train/images
val: val/images
test: test/images

nc: 5
names:
  0: longitudinal_crack
  1: transverse_crack
  2: alligator_crack
  3: other_corruption
  4: pothole
"""

with open(DATA_YAML, "w") as f:
    f.write(yaml_content)

print("data.yaml updated for Kaggle!")

data.yaml updated for Kaggle!


In [ ]:
# PHASE 3 — YOLOv8m (Freeze → Fine-Tune) [GPU T4]
from ultralytics import YOLO

DATA_YAML = "/kaggle/working/crackathon-data/data.yaml"

model_m = YOLO("yolov8m.pt")

# Phase 1: Freeze backbone
model_m.train(
    data=DATA_YAML,
    epochs=18,
    imgsz=640,
    batch=-1,
    freeze=10,
    patience=10,
    name="yolov8m_freeze",
    device=0,   # Tesla T4
    plots=False
)

In [ ]:
# YOLOv8m model
# Phase 4: Fine-tune all layers
# Reload best checkpoint explicitly
model_m = YOLO("/kaggle/working/runs/detect/yolov8m_freeze/weights/best.pt")

model_m.train(
    data=DATA_YAML,
    epochs=18,
    imgsz=640,
    batch=-1,
    freeze=0,              # unfreeze all
    patience=10,
    name="yolov8m_finetune",
    device=0,
    plots=False
)

In [13]:
# PHASE 5 — Load BEST Weights (YOLOv8m)
from ultralytics import YOLO

BEST_M = "/kaggle/working/runs/detect/yolov8m_finetune/weights/best.pt"
model_m = YOLO(BEST_M)


In [14]:
# PHASE 6 — Final Validation Check
DATA_YAML = "/kaggle/working/crackathon-data/data.yaml"

metrics = model_m.val(data=DATA_YAML)
print(metrics)

Ultralytics 8.3.244 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,842,655 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2284.9±1130.5 MB/s, size: 513.3 KB)
val: Scanning /kaggle/working/crackathon-data/randomized_dataset/val/labels.cache... 6000 images, 1792 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 6000/6000 10.1Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 375/375 2.2it/s 2:49<0.5s
                   all       6000      10443      0.609       0.55      0.584      0.312
    longitudinal_crack       2171       4093      0.614      0.526      0.552      0.309
      transverse_crack       1209       1830       0.58      0.527      0.545      0.264
       alligator_crack       1353       1698      0.623       0.58      0.632      0.342
      other_corruption       1170       1737      0.659      0.714      0.727      0.445


In [6]:
# PHASE 7 — Test-Time Augmentation (TTA) Inference
from ultralytics import YOLO
import os
import torch

MODEL_PATH = "/kaggle/working/runs/detect/yolov8m_finetune/weights/best.pt"
IMAGE_DIR = "/kaggle/working/crackathon-data/randomized_dataset/test/images"
RUN_NAME = "yolov8m_tta"

model = YOLO(MODEL_PATH)

images = sorted([
    os.path.join(IMAGE_DIR, f)
    for f in os.listdir(IMAGE_DIR)
    if f.lower().endswith((".jpg", ".png", ".jpeg"))
])

CHUNK_SIZE = 200  # safe chunk size

for i in range(0, len(images), CHUNK_SIZE):
    chunk = images[i:i + CHUNK_SIZE]
    print(f"🔄 Processing images {i} → {i+len(chunk)}")

    model.predict(
        source=chunk,
        imgsz=640,
        batch=1,
        augment=True,
        device=0 if torch.cuda.is_available() else "cpu",
        save_txt=True,
        save_conf=True,
        name=RUN_NAME,
        exist_ok=True
    )

print("✅ TTA inference completed safely")


🔄 Processing images 0 → 200

0: 640x640 (no detections), 44.1ms
1: 640x640 1 other_corruption, 44.1ms
2: 640x640 7 longitudinal_cracks, 2 transverse_cracks, 44.1ms
3: 640x640 (no detections), 44.1ms
4: 640x640 (no detections), 44.1ms
5: 640x640 1 other_corruption, 44.1ms
6: 640x640 4 longitudinal_cracks, 1 transverse_crack, 44.1ms
7: 640x640 1 alligator_crack, 1 other_corruption, 44.1ms
8: 640x640 1 transverse_crack, 44.1ms
9: 640x640 (no detections), 44.1ms
10: 640x640 (no detections), 44.1ms
11: 640x640 (no detections), 44.1ms
12: 640x640 (no detections), 44.1ms
13: 640x640 1 pothole, 44.1ms
14: 640x640 (no detections), 44.1ms
15: 640x640 (no detections), 44.1ms
16: 640x640 1 longitudinal_crack, 44.1ms
17: 640x640 (no detections), 44.1ms
18: 640x640 2 other_corruptions, 44.1ms
19: 640x640 (no detections), 44.1ms
20: 640x640 (no detections), 44.1ms
21: 640x640 (no detections), 44.1ms
22: 640x640 2 longitudinal_cracks, 1 transverse_crack, 44.1ms
23: 640x640 5 potholes, 44.1ms
24: 640x6

In [7]:
# PHASE 8 — Guaranteed Complete Post-processing
import os
import glob

# Paths
LABEL_SRC = "/kaggle/working/runs/detect/yolov8m_tta/labels"
IMAGE_DIR = "/kaggle/working/crackathon-data/randomized_dataset/test/images"
OUT_DIR = "/kaggle/working/final_predictions"

os.makedirs(OUT_DIR, exist_ok=True)

CONF_THRESH = 0.10

# Get all test image names (without extension)
image_names = [
    os.path.splitext(f)[0]
    for f in os.listdir(IMAGE_DIR)
    if f.lower().endswith((".jpg", ".png", ".jpeg"))
]

print("Total test images:", len(image_names))

for name in image_names:
    src_txt = os.path.join(LABEL_SRC, name + ".txt")
    out_txt = os.path.join(OUT_DIR, name + ".txt")

    if os.path.exists(src_txt):
        with open(src_txt) as f:
            lines = f.readlines()

        filtered = []
        for line in lines:
            cls, x, y, w, h, conf = map(float, line.split())
            if conf >= CONF_THRESH:
                filtered.append(line)

        with open(out_txt, "w") as f:
            f.writelines(filtered)

    else:
        # No detections → create empty file
        open(out_txt, "w").close()

print("Phase 8 completed: All images now have prediction files")


Total test images: 6000
Phase 8 completed: All images now have prediction files


In [8]:
# PHASE 9 — Submission ZIP (Final)
import shutil
import os

os.chdir("/kaggle/working")
shutil.make_archive("submission", "zip", "final_predictions")
print("✅ submission.zip is ready")


✅ submission.zip is ready


In [9]:
# to install runs directory
import shutil
import os

# Path to the folder you want to zip
runs_folder = "/kaggle/working/runs"

# Path for the zip file
zip_path = "/kaggle/working/runs_all_models"

# Create zip archive
shutil.make_archive(zip_path, 'zip', runs_folder)

print(f"📦 All runs zipped successfully at: {zip_path}.zip")


📦 All runs zipped successfully at: /kaggle/working/runs_all_models.zip


In [4]:
import os

ZIP_FILE = "/kaggle/working/runs_all_models.zip"

if os.path.exists(ZIP_FILE):
    os.remove(ZIP_FILE)
    print(f"✅ Deleted file: {ZIP_FILE}")
else:
    print(f"ℹ️ File not found: {ZIP_FILE}")


✅ Deleted file: /kaggle/working/runs_all_models.zip


In [ ]:
import shutil
import os

TTA_DIR = "/kaggle/working/runs_all_models.zip"

# Check if the folder exists and delete it
if os.path.exists(TTA_DIR):
    shutil.rmtree(TTA_DIR)
    print(f"✅ Deleted existing folder: {TTA_DIR}")
else:
    print(f"ℹ️ Folder does not exist: {TTA_DIR}")
